# Crawl, Load and Split KnoWhys talks from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

from datetime import datetime

from models.load_know import load_knowhy

from IPython.display import display, Markdown

from models.load_utils import Loader, load_docs_from_jsonl, save_docs_to_jsonl
from models.split_model import MarkdownSyntacticEmbeddingSplitter

## Crawl

In [ ]:
# config
host = 'https://knowhy.bookofmormoncentral.org/reference-knowhy'
source = 'knowhys'  # use this name in the directories
crawl_dir = f'../data/raw/{source}'  # base_dir is now crawl_dir
bs_parser = 'html.parser'
delay_seconds = 5

if not os.path.exists(crawl_dir):
    os.makedirs(crawl_dir)
    

In [ ]:
def extract_hrefs_from_views_field_title(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    divs_with_class = soup.find_all('div', class_='views-field-title')

    for div in divs_with_class:
        # Find all anchor tags within the div
        anchor_tags = div.find_all('a')

        # Extract href attribute from each anchor tag and append to the list
        for anchor in anchor_tags:
            href = anchor.get('href')
            if href:
                hrefs.append(urljoin(host, href))

    return hrefs

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    return os.path.join(crawl_dir, f"{path_components[-1]}.json")
    

In [ ]:
knowhy_links = []

page_counter = 0
while True:
    print(page_counter)
    status_code, html = get_page("https://knowhy.bookofmormoncentral.org/reference-knowhy?page="+str(page_counter), delay_seconds)
    if status_code != 200:
        print("Error!", status_code, page_counter)
        continue
    hrefs = extract_hrefs_from_views_field_title(html)
    if len(hrefs) == 0:
        break
    else:
        knowhy_links.extend(hrefs)
        page_counter += 1

In [ ]:
for url in knowhy_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")

## Load

In [ ]:
# config
# input_dir is now crawl_dir, and output_dir is now load_dir, and output_filename is now load_filename
load_dir = f'../data/load/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
load_filename = os.path.join(load_dir, f"{today}.jsonl")

if not os.path.exists(load_dir):
    os.makedirs(load_dir)

In [ ]:
loader = Loader(load_knowhy, crawl_dir)
docs = loader.load(verbose=True)
len(docs)

In [ ]:
save_docs_to_jsonl(docs, load_filename)

# Split loaded documents

In [ ]:
# configure
# input_path is now load_filename, output_dir is now split_dir, and output filename is now split_filename
split_dir = f'../data/split/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
# output filename is now split_filename
split_filename = os.path.join(split_dir, f"{today}.jsonl")

if not os.path.exists(split_dir):
    os.makedirs(split_dir)

## Load documents

In [ ]:
docs = load_docs_from_jsonl(load_filename)
len(docs)

## Create splits

In [ ]:
text_splitter = MarkdownSyntacticEmbeddingSplitter()

In [ ]:
splits = text_splitter.split_documents(docs, verbose=True)
len(splits)

In [ ]:
for ix, split in enumerate(splits[:10]):
    print(ix, split.metadata["url"])
    print(split.page_content)
    print("\n!!! SPLIT !!!\n")

## Save splits 

In [ ]:
save_docs_to_jsonl(splits, split_filename)